In [ ]:
!pip install torch==2.0.0
!pip install transformers==4.35.0
!pip install sentence-transformers==2.2.2
!pip install huggingface-hub==0.25.2
!pip install faiss-cpu==1.7.4


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

embedder = SentenceTransformer("all-MiniLM-L6-v2")
print("✅ Embedding model loaded!")

model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
print("✅ Dialogue model loaded!")


In [ ]:
import pandas as pd
import numpy as np
import json
import warnings
import re
from datetime import datetime
from typing import List, Dict, Any
import gradio as gr

# Skip sentence-transformers and use simple text similarity instead
USE_TRANSFORMERS = True
USE_EMBEDDINGS = False  # Skip this since sentence-transformers isn't working
USE_FAISS = False      # Skip this too to avoid complications

warnings.filterwarnings('ignore')
print("✅ Basic libraries imported successfully!")
print("ℹ️  Using rule-based approach (more reliable for demo)")


In [ ]:
synthetic_patients = [
    {
        "patient_id": "P001",
        "demographics": "45-year-old male",
        "conditions": "Hypertension, Diabetes",
        "symptoms": "chest pain, shortness of breath, sweating",
        "vitals": "BP: 160/95, HR: 105, Temp: 98.6F",
        "medications": "Metformin, Lisinopril",
        "risk_factors": "smoker, family history of heart disease"
    },
    {
        "patient_id": "P002",
        "demographics": "28-year-old female",
        "conditions": "None",
        "symptoms": "fever, headache, neck stiffness, photophobia",
        "vitals": "BP: 110/70, HR: 120, Temp: 102.5F",
        "medications": "None",
        "risk_factors": "None"
    },
    {
        "patient_id": "P003",
        "demographics": "65-year-old female",
        "conditions": "Diabetes, CKD",
        "symptoms": "shortness of breath, swollen legs, fatigue",
        "vitals": "BP: 140/90, HR: 90, O2: 88%",
        "medications": "Metformin, Furosemide",
        "risk_factors": "diabetic for 20 years"
    }
]

medical_qa = [
    {
        "question": "What are the signs of acute myocardial infarction?",
        "answer": "Chest pain radiating to arm/jaw, diaphoresis, nausea, shortness of breath, ST elevation on ECG"
    },
    {
        "question": "What is the treatment for suspected meningitis?",
        "answer": "Immediate antibiotics (ceftriaxone), lumbar puncture, blood cultures, dexamethasone if bacterial"
    },
    {
        "question": "How do you manage heart failure exacerbation?",
        "answer": "Diuretics, ACE inhibitors, beta blockers, oxygen support, daily weights monitoring"
    },
    {
        "question": "What are contraindications for ACE inhibitors?",
        "answer": "Pregnancy, bilateral renal artery stenosis, hyperkalemia, angioedema history"
    }
]

print(f"✅ Loaded {len(synthetic_patients)} synthetic patients")
print(f"✅ Loaded {len(medical_qa)} medical Q&A pairs")


In [ ]:
def create_medical_knowledge_base(patients, qa_data):
    """Create searchable medical knowledge base from datasets"""
    knowledge_base = []


    for patient in patients:
        case_text = f"""
        Patient Case: {patient['demographics']}
        Medical History: {patient['conditions']}
        Presenting Symptoms: {patient['symptoms']}
        Vital Signs: {patient['vitals']}
        Current Medications: {patient['medications']}
        Risk Factors: {patient['risk_factors']}
        """
        knowledge_base.append(case_text.strip())

    for qa in qa_data:
        qa_text = f"Medical Question: {qa['question']}\nMedical Answer: {qa['answer']}"
        knowledge_base.append(qa_text)

    clinical_guidelines = [
        "STEMI Management: Door-to-balloon time <90 minutes, dual antiplatelet therapy, primary PCI preferred",
        "Sepsis Protocol: Antibiotics within 1 hour, fluid resuscitation, lactate measurement, blood cultures",
        "Heart Failure: ACE inhibitors, beta blockers, diuretics, daily weight monitoring",
        "Diabetes Management: HbA1c target <7%, metformin first line, insulin if inadequate control"
    ]

    knowledge_base.extend(clinical_guidelines)

    return knowledge_base

knowledge_base = create_medical_knowledge_base(synthetic_patients, medical_qa)
print(f"✅ Created knowledge base with {len(knowledge_base)} entries")


In [ ]:
# RAG
import faiss
import numpy as np

print("🔄 Creating embeddings...")
embeddings = embedder.encode(knowledge_base)

# Create FAISS vector index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings.astype('float32'))

print(f"✅ RAG system ready with {index.ntotal} documents")

def retrieve_relevant_context(query: str, top_k: int = 5) -> List[str]:
    """Retrieve relevant medical context for query"""
    query_embedding = embedder.encode([query])
    scores, indices = index.search(query_embedding.astype('float32'), top_k)

    relevant_docs = []
    for i, idx in enumerate(indices[0]):
        doc = knowledge_base[idx]
        score = scores[0][i]
        relevant_docs.append(f"[Relevance: {score:.2f}] {doc}")

    return relevant_docs

# Test retrieval
test_query = "chest pain radiating to arm"
context = retrieve_relevant_context(test_query)
print(f"✅ RAG system test successful - retrieved {len(context)} relevant documents")


In [ ]:
class ClinicalSafetyEngine:
    """Rule-based safety alert system"""

    def __init__(self):
        self.critical_patterns = {
            'cardiac_emergency': [
                r'chest pain.*radiating',
                r'st elevation',
                r'stemi',
                r'myocardial infarction',
                r'heart attack'
            ],
            'neurological_emergency': [
                r'neck stiffness.*photophobia',
                r'meningitis',
                r'stroke',
                r'seizure'
            ],
            'respiratory_emergency': [
                r'shortness of breath.*o2.*\d+%',
                r'respiratory distress',
                r'pneumothorax'
            ],
            'sepsis': [
                r'fever.*hypotension',
                r'sepsis',
                r'septic shock'
            ]
        }

    def assess_urgency(self, clinical_text: str) -> Dict[str, Any]:
        """Assess clinical urgency and generate alerts"""
        clinical_text_lower = clinical_text.lower()

        alerts = {
            'risk_level': 'LOW',
            'urgency_score': 0,
            'alerts': [],
            'immediate_actions': []
        }

        for category, patterns in self.critical_patterns.items():
            for pattern in patterns:
                if re.search(pattern, clinical_text_lower):
                    if category == 'cardiac_emergency':
                        alerts['risk_level'] = 'CRITICAL'
                        alerts['urgency_score'] = 10
                        alerts['alerts'].append('🚨 SUSPECTED CARDIAC EMERGENCY')
                        alerts['immediate_actions'].extend([
                            'Obtain 12-lead ECG STAT',
                            'Administer aspirin 325mg if no contraindications',
                            'Activate cardiac catheterization lab',
                            'IV access and continuous monitoring'
                        ])
                    elif category == 'neurological_emergency':
                        alerts['risk_level'] = 'CRITICAL'
                        alerts['urgency_score'] = 10
                        alerts['alerts'].append('🚨 SUSPECTED NEUROLOGICAL EMERGENCY')
                        alerts['immediate_actions'].extend([
                            'Lumbar puncture consideration',
                            'Blood cultures before antibiotics',
                            'Empirical antibiotics if meningitis suspected',
                            'Isolation precautions'
                        ])

        return alerts

safety_engine = ClinicalSafetyEngine()
print("✅ Clinical Safety Engine initialized")


In [ ]:
class HealthcareCopilot:
    """Main Healthcare Copilot system"""

    def __init__(self, model, tokenizer, embedder, index, knowledge_base, safety_engine):
        self.model = model
        self.tokenizer = tokenizer
        self.embedder = embedder
        self.index = index
        self.knowledge_base = knowledge_base
        self.safety_engine = safety_engine

    def extract_clinical_features(self, text: str) -> Dict[str, Any]:
        """Extract key clinical features from input text"""
        features = {
            'age': None,
            'gender': None,
            'symptoms': [],
            'vitals': {},
            'medications': []
        }


        age_match = re.search(r'(\d+)-year-old', text.lower())
        if age_match:
            features['age'] = int(age_match.group(1))

        gender_match = re.search(r'(male|female)', text.lower())
        if gender_match:
            features['gender'] = gender_match.group(1)


        bp_match = re.search(r'bp:?\s*(\d+/\d+)', text.lower())
        if bp_match:
            features['vitals']['blood_pressure'] = bp_match.group(1)

        hr_match = re.search(r'hr:?\s*(\d+)', text.lower())
        if hr_match:
            features['vitals']['heart_rate'] = int(hr_match.group(1))

        return features

    def generate_differential_diagnosis(self, clinical_input: str, context: List[str]) -> Dict[str, Any]:
        """Generate differential diagnosis with confidence scores"""

        diagnoses = []

        clinical_lower = clinical_input.lower()

        # Cardiac conditions
        cardiac_score = 0
        if any(term in clinical_lower for term in ['chest pain', 'radiating', 'sweating', 'nausea']):
            cardiac_score += 30
        if 'st elevation' in clinical_lower:
            cardiac_score += 50
        if any(term in clinical_lower for term in ['hypertension', 'diabetes', 'smoker']):
            cardiac_score += 20

        if cardiac_score > 0:
            diagnoses.append({
                'condition': 'Acute Coronary Syndrome',
                'confidence': min(cardiac_score, 95),
                'category': 'Cardiac Emergency'
            })

        # Infectious conditions
        infection_score = 0
        if any(term in clinical_lower for term in ['fever', 'headache', 'neck stiffness']):
            infection_score += 40
        if 'photophobia' in clinical_lower:
            infection_score += 30

        if infection_score > 0:
            diagnoses.append({
                'condition': 'Meningitis',
                'confidence': min(infection_score, 90),
                'category': 'Infectious Emergency'
            })

        # Sort by confidence
        diagnoses.sort(key=lambda x: x['confidence'], reverse=True)

        return {'diagnoses': diagnoses}

    def generate_treatment_recommendations(self, clinical_input: str, diagnoses: List[Dict]) -> List[str]:
        """Generate evidence-based treatment recommendations"""
        recommendations = []

        if diagnoses:
            top_diagnosis = diagnoses[0]

            if 'coronary' in top_diagnosis['condition'].lower():
                recommendations.extend([
                    "🫀 **Cardiac Protocol:**",
                    "• Aspirin 325mg chewed immediately",
                    "• 12-lead ECG within 10 minutes",
                    "• IV access and continuous monitoring",
                    "• Clopidogrel 600mg loading dose",
                    "• Prepare for primary PCI if STEMI"
                ])

            elif 'meningitis' in top_diagnosis['condition'].lower():
                recommendations.extend([
                    "🧠 **Meningitis Protocol:**",
                    "• Blood cultures before antibiotics",
                    "• Ceftriaxone 2g IV STAT",
                    "• Dexamethasone 10mg IV if bacterial suspected",
                    "• Lumbar puncture (if no contraindications)",
                    "• Isolation precautions"
                ])

        return recommendations

    def analyze_patient(self, clinical_input: str) -> Dict[str, Any]:
        """Main analysis function"""
        # Extract clinical features
        features = self.extract_clinical_features(clinical_input)

        # Retrieve relevant context
        context = retrieve_relevant_context(clinical_input, top_k=5)

        # Safety assessment
        safety_alerts = self.safety_engine.assess_urgency(clinical_input)

        # Generate differential diagnosis
        diagnosis_results = self.generate_differential_diagnosis(clinical_input, context)

        # Generate treatment recommendations
        treatment_recs = self.generate_treatment_recommendations(clinical_input, diagnosis_results['diagnoses'])

        # Compile final response
        response = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'clinical_features': features,
            'safety_alerts': safety_alerts,
            'differential_diagnosis': diagnosis_results['diagnoses'],
            'treatment_recommendations': treatment_recs,
            'relevant_context': context[:3],  # Top 3 most relevant
            'confidence_score': max([d['confidence'] for d in diagnosis_results['diagnoses']]) if diagnosis_results['diagnoses'] else 0
        }

        return response

# Initialize the copilot
copilot = HealthcareCopilot(model, tokenizer, embedder, index, knowledge_base, safety_engine)
print("✅ Healthcare Copilot initialized successfully!")


In [ ]:
def format_copilot_response(response: Dict[str, Any]) -> str:
    """Format the copilot response for display"""

    output = f"""
# 🏥 Healthcare Copilot Analysis
**Timestamp:** {response['timestamp']}

## 🚨 Safety Assessment
**Risk Level:** {response['safety_alerts']['risk_level']}
**Urgency Score:** {response['safety_alerts']['urgency_score']}/10

"""

    if response['safety_alerts']['alerts']:
        output += "### Critical Alerts:\n"
        for alert in response['safety_alerts']['alerts']:
            output += f"- {alert}\n"
        output += "\n"

    if response['safety_alerts']['immediate_actions']:
        output += "### 🚨 Immediate Actions Required:\n"
        for action in response['safety_alerts']['immediate_actions']:
            output += f"- {action}\n"
        output += "\n"

    output += f"## 📋 Differential Diagnosis\n"
    if response['differential_diagnosis']:
        for i, diagnosis in enumerate(response['differential_diagnosis'][:3], 1):
            output += f"{i}. **{diagnosis['condition']}** (Confidence: {diagnosis['confidence']}%)\n"
            output += f"   - Category: {diagnosis['category']}\n"
    else:
        output += "No specific diagnosis identified. Consider comprehensive evaluation.\n"

    output += f"\n## 💊 Treatment Recommendations\n"
    if response['treatment_recommendations']:
        for rec in response['treatment_recommendations']:
            output += f"{rec}\n"
    else:
        output += "Consult clinical guidelines for appropriate management.\n"

    output += f"\n## 📊 Clinical Features Detected\n"
    features = response['clinical_features']
    if features['age']:
        output += f"- **Age:** {features['age']} years\n"
    if features['gender']:
        output += f"- **Gender:** {features['gender']}\n"
    if features['vitals']:
        output += f"- **Vitals:** {features['vitals']}\n"

    output += f"\n**Overall Confidence:** {response['confidence_score']}%"

    return output

def analyze_patient_gradio(clinical_input: str) -> str:
    """Gradio interface function"""
    if not clinical_input.strip():
        return "Please enter patient information to analyze."

    try:
        response = copilot.analyze_patient(clinical_input)
        return format_copilot_response(response)
    except Exception as e:
        return f"Error processing request: {str(e)}"

# Create Gradio interface
demo = gr.Interface(
    fn=analyze_patient_gradio,
    inputs=[
        gr.Textbox(
            lines=5,
            placeholder="Enter patient information:\n\nExample: 45-year-old male presents with severe chest pain for 2 hours, pain radiating to left arm, diaphoresis, BP: 160/95, HR: 110, smoker for 15 years",
            label="Clinical Input"
        )
    ],
    outputs=[
        gr.Markdown(label="Copilot Analysis")
    ],
    title="🏥 Healthcare Copilot - Clinical Decision Support",
    description="AI-powered clinical decision support system for healthcare professionals. Enter patient symptoms and clinical data to receive evidence-based diagnostic and treatment recommendations.",
    # examples=[
    #     ["45-year-old male, chest pain radiating to left arm, diaphoresis, BP: 160/95, HR: 110, smoker"],
    #     ["28-year-old female, fever for 3 days, severe headache, neck stiffness, photophobia, temp: 102.5F"],
    #     ["65-year-old female with diabetes, shortness of breath, swollen legs, fatigue, BP: 140/90, O2 sat: 88%"]
    # ]
)

print("✅ Gradio interface created successfully!")


In [ ]:

demo.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

print("🚀 Healthcare Copilot is now running!")
print("📱 Use the public URL to access your copilot from anywhere")
